In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [82]:
CAMBIO_REGIONES = {
    "Metropolitana de Santiago": "Metropolitana de Santiago",
    "de Valparaíso": "Valparaíso",
    "del Biobío": "Biobío",
    "del Libertador B. O'Higgins": "Libertador General Bernardo O'Higgins",
    "de La Araucanía": "La Araucanía",
    "de Los Lagos": "Los Lagos",
    "del Maule": "Maule",
    "de Ñuble": "Ñuble",
    "de Coquimbo": "Coquimbo",
    "de Los Rios": "Los Ríos",
    "de Magallanes y de la Antártica Chilena": "Magallanes y de la Antártica Chilena",
    "de Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    "de Antofagasta": "Antofagasta",
    "de Atacama": "Atacama",
    "de Tarapacá": "Tarapacá",
    "de Arica y Parinacota": "Arica y Parinacota",
}

In [9]:
# Lee el diccionario de regiones
diccionario_regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name="Anexo 2", header=6)
    .iloc[:, 5:9]
    .dropna(subset="Código Comuna")
)

diccionario_regiones["Código Comuna"] = (
    diccionario_regiones["Código Comuna"].astype(int).astype(str)
)

In [51]:
df = pd.read_excel(
    "../data/raw/1_poblacion_ine/censo_2024/DATOS COMUNA PARA POBLACION POR EDAD 25.06.2025.xlsx"
)

In [52]:
df["CÓDIGO COMUNA"] = df["CÓDIGO COMUNA"].astype(str)
df = df.merge(diccionario_regiones, how="left", left_on="CÓDIGO COMUNA", right_on="Código Comuna")

# Elimina el registro del pais
df = df.query("`NOMBRE COMUNA` != 'PAÍS'")

In [53]:
columnas_hombres = df.columns[df.columns.str.contains("HOMBRES")]
columnas_mujeres = df.columns[df.columns.str.contains("MUJERES")]
columnas_valores = list(columnas_hombres) + list(columnas_mujeres)

In [83]:
# Cambia a formato largo
formato_largo_poblacion = pd.melt(
    df,
    id_vars=["Código Región", "Nombre Región", "Código Comuna", "Nombre Comuna", "Edad"],
    value_vars=columnas_valores,
).rename(columns={"variable": "hombre_mujer"})


# Agrega el valor de hombre y mujer
formato_largo_poblacion[["hombre_mujer", "ano"]] = formato_largo_poblacion[
    "hombre_mujer"
].str.split(" ", expand=True)

# Cambia a formato antiguo
formato_largo_poblacion["hombre_mujer"] = formato_largo_poblacion["hombre_mujer"].replace(
    {"HOMBRES": "1", "MUJERES": "2"}
)

# Convierte los datos a formato ancho
formato_ancho_poblacion = pd.pivot_table(
    formato_largo_poblacion,
    index=[
        "Código Región",
        "Nombre Región",
        "Código Comuna",
        "Nombre Comuna",
        "Edad",
        "hombre_mujer",
    ],
    columns="ano",
    values="value",
    aggfunc="sum",
).reset_index()

# Cambia el nombre de las columnas
formato_ancho_poblacion = formato_ancho_poblacion.rename(
    columns={"Código Región": "Region", "Nombre Región": "M", "Código Comuna": "Comuna"}
).reset_index(drop=True)

# Cambia las glosas de las regiones
formato_ancho_poblacion["M"] = formato_ancho_poblacion["M"].replace(CAMBIO_REGIONES)